In [1]:
from sklearn.cluster import dbscan
import pickle
import numpy as np
import folium 
import math
import matplotlib.pyplot as plt

In [2]:
datapath = "../attack_data/gps_seqs"

In [3]:
with open(datapath+"/traingps", "rb") as f:
    traingps = pickle.load(f)

with open(datapath+"/valgps", "rb") as f:
    valgps = pickle.load(f)

In [4]:
with open(datapath + "/testgps", "rb") as f:
    testgps = pickle.load(f)

In [5]:
# 由于GPS坐标是球面坐标，所以需要将gps坐标转换为web墨卡托坐标
def lonlat2meters(lon, lat):
    semimajoraxis = 6378137.0
    east = lon * 0.017453292519943295
    north = lat * 0.017453292519943295
    t = np.sin(north)
    return semimajoraxis * east, 3189068.5 * np.log((1 + t) / (1 - t))

# 聚类后的结果如果需要使用，还需要转换为GPS坐标
def meters2lonlat(x, y):
    semimajoraxis = 6378137.0
    lon = x / semimajoraxis / 0.017453292519943295
    t = math.exp(y / 3189068.5)
    lat = math.asin((t - 1) / (t + 1)) / 0.017453292519943295
    return lon, lat

In [6]:
# 使用十万条轨迹进行聚类
testData = traingps[0:100000]

In [7]:
allPoints = []
for item in testData:
    allPoints.extend(list(item))
allPoints = [list(i) for i in allPoints]

In [8]:
allPointsXY = [lonlat2meters(i[0], i[1]) for i in allPoints]
allPointsXY = np.array(allPointsXY)
print(len(allPointsXY))

4694747


In [9]:
print(np.unique(allPointsXY).shape)

(48678,)


In [10]:
''' 这四行代码好像别的地方没用到，不知道干啥用的 '''
# kkk = []
# ddd = (((allPointsXY[1]-allPointsXY)**2).sum(axis=1)**0.5)
# ddd.sort()
# kkk.append(ddd[3])

' 这四行代码好像别的地方没用到，不知道干啥用的 '

In [12]:
# 50 100有140条
# 50 3000 -> 105
# 50 4000 -> 97
''' ****之后可能需要修改为python文件， jupyter kernel这一部分代码跑不动**** '''
'''
 core_samples: ndarray of shape (n_core_samples,), Indices of core samples.
 labels: ndarray of shape (n_samples,) Cluster labels for each point. Noisy samples are given the label -1. 
'''
core_samples,cluster_ids = dbscan(allPointsXY, eps = 50, min_samples= 4000)
print(cluster_ids)

: 

In [10]:
print(len(cluster_ids))

4694747


In [11]:
# cluster_ids -> set
clusterSet = set(cluster_ids)
clusterSet.remove(-1)
print(clusterSet)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96}


In [12]:
# 统计没有归入聚类的gps点数量
count = 0
for id in cluster_ids:
    if(id == -1):
        count += 1
print(count)

2823175


In [13]:
meanXY = {}
meanGPS = {}
for id in clusterSet:
    count = 0
    sumXY = np.array([0.0,0.0])
    for i in range(len(allPointsXY)):
        if(cluster_ids[i] == id):
            count += 1
            sumXY += np.array(allPointsXY[i])

    meanXY[id] = sumXY / count
    meanGPS[id] = meters2lonlat(meanXY[id][0], meanXY[id][1])    # longtitude, latitude

In [14]:
print(meanXY)
print(meanGPS)

{0: array([-960237.00525294, 5036993.35436517]), 1: array([-962014.33105234, 5035982.66017588]), 2: array([-958619.57498907, 5033997.03984333]), 3: array([-960155.01397773, 5034485.24028332]), 4: array([-963458.746654  , 5038609.43010536]), 5: array([-961422.0664943 , 5036600.25280868]), 6: array([-960759.18379888, 5035197.32863071]), 7: array([-960334.00923714, 5035163.19016142]), 8: array([-958832.24389309, 5035095.5507809 ]), 9: array([-957400.96626459, 5034304.69519546]), 10: array([-957580.2752281 , 5034082.02857298]), 11: array([-960096.34083312, 5035938.4542577 ]), 12: array([-960574.1655346 , 5035648.13891152]), 13: array([-960169.93554316, 5035568.91602943]), 14: array([-959476.23020804, 5035275.84688416]), 15: array([-958680.87871627, 5035039.18175711]), 16: array([-958441.16837946, 5034823.1496678 ]), 17: array([-957996.33243825, 5035004.17573398]), 18: array([-958125.44704574, 5034541.38373975]), 19: array([-957189.88343958, 5034272.683615  ]), 20: array([-956637.1246791 , 

In [15]:
''' 虽然写的是直径，但算的其实是半径 '''
diameters = {}
for id in clusterSet:
    d = 0
    for i in range(len(allPointsXY)):
        if(cluster_ids[i] == id):
            d0 = np.linalg.norm(allPointsXY[i]-meanXY[id])
            if(d0 > d):
                d = d0
    diameters[id] = d

In [16]:
print(len(diameters.values()))
print(diameters[1])

97
370.6852652261462


In [17]:
clusters = {}
clusters["meanXY"] = meanXY
clusters["meanGPS"] = meanGPS
clusters["diameters"] = diameters
with open("./data/clusters", "wb") as f:
    pickle.dump(clusters, f)

In [10]:
with open("./data/clusters", "rb") as f:
    clusters = pickle.load(f)

meanXY = clusters["meanXY"]
meanGPS = clusters["meanGPS"]
diameters = clusters["diameters"]
clusterSet = list(range(97))

In [11]:
my_map = folium.Map(zoom_start = 15)
# for i in range(len(allPoints)):
#     if(cluster_ids[i]%2 == 0):
#         folium.Circle( radius = 10,
#                             location=[allPoints[i][1],allPoints[i][0]],
#                             color='red',
#                             opacity = 1,
#                             fill=False,
#                             fill_opacity = 1
#                             ).add_to(my_map)
#     if(cluster_ids[i]%2 == 1):
#         folium.Circle( radius = 10,
#                             location=[allPoints[i][1],allPoints[i][0]],
#                             color='green',
#                             opacity = 1,
#                             fill=False,
#                             fill_opacity = 1
#                             ).add_to(my_map)

for id in clusterSet:
    folium.Circle( radius = 100,
                            location=[meanGPS[id][1],meanGPS[id][0]],
                            color='red',
                            opacity = 1,
                            fill=False,
                            fill_opacity = 1
                            ).add_to(my_map)
my_map

In [12]:
# 获取每条轨迹经过的ROI区域编号
# 即轨迹上的所有点，如果其在任意一个ROI的范围内，则认为该轨迹经过了这个ROI
trainData = traingps[0:180000]
trainDataXY = []
for i in range(len(trainData)):
    xyList = []
    for j in range(trainData[i].shape[0]):
        x, y = lonlat2meters(trainData[i][j][0], trainData[i][j][1])
        coord = np.array([x,y])    # 这里的coord是web墨卡托坐标
        xyList.append(coord)
    xyList = np.array(xyList)
    trainDataXY.append(xyList)

In [13]:
cluster_radius = 500

In [14]:
from concurrent.futures import ProcessPoolExecutor
import pickle

def getLabel(i):    #在这里传入参数只有index, trainDataXY是函数内获取的全局变量
    label = np.zeros(len(clusterSet))
    for j in range(trainDataXY[i].shape[0]):
        for id in clusterSet:
            # print(j, id)
            if(np.linalg.norm(meanXY[id]-trainDataXY[i][j]) < cluster_radius):
                label[id] = 1
    return label

iRange = list(range(len(trainDataXY)))
with ProcessPoolExecutor(max_workers=30) as executor:
    trainResults = executor.map(getLabel, iRange)

trainLabelSet = list(trainResults)


In [15]:
trainLabelSet = np.array(trainLabelSet)

In [16]:
labelNum = [int(i) for i in sum(trainLabelSet)]
print(labelNum)
print(sum(labelNum))

[14836, 18136, 57827, 34691, 14951, 17257, 37315, 43123, 33700, 23385, 23395, 25324, 27857, 27775, 16389, 34980, 44017, 35001, 48975, 19643, 14901, 17654, 21649, 24368, 21083, 10355, 16302, 34633, 15700, 17846, 23046, 13334, 17022, 17293, 14664, 16162, 27901, 23108, 13700, 12107, 13644, 24840, 26711, 31092, 29020, 28789, 32608, 21466, 30974, 14200, 40284, 27147, 32551, 6653, 28161, 17143, 21147, 31050, 35049, 27118, 10578, 24042, 13833, 17502, 11907, 10986, 10429, 18801, 17918, 23314, 19053, 16588, 6334, 35442, 16979, 16745, 6203, 13441, 40511, 33292, 19831, 35311, 10433, 29188, 17704, 10075, 23451, 16038, 15933, 30934, 32331, 14432, 24089, 15842, 8085, 15116, 25981]
2199724


In [17]:
with open("./data/r{}_label_train".format(cluster_radius), "wb") as f:
    pickle.dump(trainLabelSet, f)

In [18]:
# 获取每条轨迹经过的ROI区域编号
# 即轨迹上的所有点，如果其在任意一个ROI的范围内，则认为该轨迹经过了这个ROI
valData = valgps
valDataXY = []
for i in range(len(valData)):
    xyList = []
    for j in range(valData[i].shape[0]):
        x, y = lonlat2meters(valData[i][j][0], valData[i][j][1])
        coord = np.array([x,y])
        xyList.append(coord)
    xyList = np.array(xyList)
    valDataXY.append(xyList)

from concurrent.futures import ProcessPoolExecutor
import pickle

def getLabel(i):
    label = np.zeros(len(diameters))
    for j in range(valDataXY[i].shape[0]):
        for id in clusterSet:
            # print(j, id)
            if(np.linalg.norm(meanXY[id]-valDataXY[i][j]) < cluster_radius):
                label[id] = 1
    return label

iRange = list(range(len(valDataXY)))
with ProcessPoolExecutor(max_workers=30) as executor:
    valResults = executor.map(getLabel, iRange)


In [19]:
valLabelSet = list(valResults)

In [20]:
labelNum = [int(i) for i in sum(valLabelSet)]
print(labelNum)
print(sum(labelNum))

[819, 1025, 2664, 1732, 690, 948, 2029, 2398, 1905, 1258, 1225, 1454, 1693, 1562, 882, 1972, 2439, 1929, 2599, 1054, 857, 1213, 1174, 1297, 1113, 554, 983, 1554, 730, 1056, 1287, 757, 925, 1199, 862, 879, 1574, 1325, 812, 719, 811, 1192, 1522, 1820, 1704, 1680, 1907, 1306, 1753, 903, 1730, 1826, 1849, 462, 1322, 941, 1276, 1816, 2043, 1799, 642, 1357, 722, 959, 933, 849, 828, 1056, 982, 1475, 1035, 942, 275, 2070, 952, 957, 316, 804, 2108, 1707, 1105, 1693, 677, 1556, 1223, 804, 1466, 912, 878, 1511, 1727, 781, 1265, 958, 412, 669, 1315]
122730


In [21]:
valLabelSet = np.array(valLabelSet)
with open("./data/r{}_label_val".format(cluster_radius), "wb") as f:
    pickle.dump(valLabelSet, f)

In [22]:
# 获取每条轨迹经过的ROI区域编号
# 即轨迹上的所有点，如果其在任意一个ROI的范围内，则认为该轨迹经过了这个ROI
testData = testgps
testDataXY = []
for i in range(len(testData)):
    xyList = []
    for j in range(testData[i].shape[0]):
        x, y = lonlat2meters(testData[i][j][0], testData[i][j][1])
        coord = np.array([x,y])
        xyList.append(coord)
    xyList = np.array(xyList)
    testDataXY.append(xyList)

from concurrent.futures import ProcessPoolExecutor
import pickle

def getLabel(i):
    label = np.zeros(len(diameters))
    for j in range(testDataXY[i].shape[0]):
        for id in clusterSet:
            # print(j, id)
            if(np.linalg.norm(meanXY[id]-testDataXY[i][j]) < cluster_radius):
                label[id] = 1
    return label

iRange = list(range(len(testDataXY)))
with ProcessPoolExecutor(max_workers=30) as executor:
    testResults = executor.map(getLabel, iRange)


In [23]:
testLabelSet = list(testResults)

In [24]:
labelNum = [int(i) for i in sum(testLabelSet)]
print(labelNum)
print(sum(labelNum))

[789, 1077, 4217, 2469, 849, 888, 2566, 2797, 1970, 1242, 1282, 1309, 1478, 1462, 886, 2034, 2588, 2063, 2840, 998, 714, 884, 1469, 1746, 1594, 746, 989, 2406, 1134, 1101, 1332, 728, 919, 861, 777, 822, 1483, 1333, 735, 668, 624, 1434, 1438, 1640, 1448, 1472, 1741, 1287, 1863, 857, 2587, 1302, 1873, 147, 1605, 911, 1276, 1643, 1953, 1334, 620, 1382, 747, 934, 500, 488, 430, 1093, 1057, 1148, 974, 835, 395, 1967, 894, 839, 364, 609, 2424, 1888, 1002, 2378, 583, 1560, 892, 370, 1300, 818, 964, 1830, 2334, 744, 1654, 872, 564, 774, 1486]
126393


In [25]:
testLabelSet = np.array(testLabelSet)
with open("./data/r{}_label_test".format(cluster_radius), "wb") as f:
    pickle.dump(testLabelSet, f)